<a href="https://colab.research.google.com/github/gaheehan/NLP/blob/main/Transfomer_%EC%B1%97%EB%B4%87_%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfomer 챗봇 만들기

## Project Description


참고: https://velog.io/@seolini43/%ED%8C%8C%EC%9D%B4%EC%8D%ACTransformer%EB%A1%9C-%EC%98%A4%ED%94%BC%EC%8A%A4-%EC%B1%97%EB%B4%87-%EB%A7%8C%EB%93%A4%EA%B8%B0-%EC%BD%94%EB%93%9C <br>
Transformer를 이용한 이번 프로젝트는 AIHUB의 한국어 대화 모델 세트 중 카페 데이터셋을 다운로드하여 카페 키오스크 챗봇을 만들어 보는 것이다. 

코드는 https://wikidocs.net/31379 와,위의 블로그를 참고했고, 데이터 적용코드는 변형하였으며, 전체적으로 트랜스포머를 구현하는 코드는 거의 같다.

코드는 파이썬으로 Colab에서 작성하였고, GPU를 사용하였다.

그 외 환경

*   batch size = 64
*   buffer size = 20000
*   epochs = 50

## 1.환경설정 & 데이터 준비

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import time
import tensorflow_datasets as tfds
import tensorflow as tf

In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 데이터 불러오기

In [ ]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/F 카페(7,859)_new.xlsx')
df.head()

,SPEAKER,SENTENCE,DOMAINID,DOMAIN,CATEGORY,SPEAKERID,SENTENCEID,MAIN,SUB,QA,QACNCT,MQ,SQ,UA,SA,개체명,용어사전,지식베이스
0,고객,아이스아메리카노 하나요,F,카페,카페,1.0,1.0,일반주문,NaN,Q,NaN,아이스 아메리카노 하나요,NaN,NaN,NaN,"아이스 아메리카노, 하나",NaN,NaN
1,점원,테이크아웃하실 건가요?,F,카페,카페,0.0,2.0,일반주문,테이크아웃여부,Q,NaN,NaN,테이크아웃하실 건가요?,NaN,NaN,테이크아웃,NaN,NaN
2,고객,네 텀블러에 넣어주세요,F,카페,카페,1.0,3.0,텀블러사용,NaN,Q,NaN,네 텀블러에 넣어주세요,NaN,NaN,NaN,텀블러,NaN,NaN
3,점원,텀블러 할인 300원 해드릴게요,F,카페,카페,0.0,4.0,텀블러사용,NaN,A,NaN,NaN,NaN,NaN,텀블러 할인 300원 해드릴게요,"텀블러, 할인, 300원",NaN,NaN
4,고객,그란데 사이즈로 주세요,F,카페,카페,1.0,5.0,일반주문,NaN,Q,NaN,그란데 사이즈로 주세요,NaN,NaN,NaN,그란데 사이즈,NaN,NaN


### 필요한 데이터만 추출하기

In [ ]:
df = df[['SPEAKER', 'SENTENCE']]
df

,SPEAKER,SENTENCE
0,고객,아이스아메리카노 하나요
1,점원,테이크아웃하실 건가요?
2,고객,네 텀블러에 넣어주세요
3,점원,텀블러 할인 300원 해드릴게요
4,고객,그란데 사이즈로 주세요
...,...,...
7854,고객,아이스 아메리카노는 가격이 다른가요?
7855,고객,텀블러 할인하면 얼마인가요?
7856,고객,총 가격이 얼마입니까?
7857,고객,다 합쳐서 가격이 얼마인 거죠?


In [ ]:
df_Q = df[df['SPEAKER']== '고객']
df_Q

,SPEAKER,SENTENCE
0,고객,아이스아메리카노 하나요
2,고객,네 텀블러에 넣어주세요
4,고객,그란데 사이즈로 주세요
6,고객,네 보고쿠폰으로 결제할게요
8,고객,괜찮습니다
...,...,...
7854,고객,아이스 아메리카노는 가격이 다른가요?
7855,고객,텀블러 할인하면 얼마인가요?
7856,고객,총 가격이 얼마입니까?
7857,고객,다 합쳐서 가격이 얼마인 거죠?


In [ ]:
df_A = df[df['SPEAKER']== '점원']
df_A

,SPEAKER,SENTENCE
1,점원,테이크아웃하실 건가요?
3,점원,텀블러 할인 300원 해드릴게요
5,점원,적립카드 있으세요?
7,점원,현금영수증 해드릴까요?
9,점원,저쪽에서 잠시만 대기해주세요
...,...,...
7171,점원,가지고 가시면 2000원이에요
7173,점원,드시고 가시면 3000원이에요
7175,점원,현금 영수증 맞으세요?
7177,점원,"네, 여기 드릴게요"


In [ ]:
print(df_Q.isnull().values.any())
print(df_A.isnull().values.any())

False
False


In [ ]:
print(len(df_Q),len(df_A))

4274 3585


In [ ]:
df_Q = df_Q[:3585]
print(len(df_Q),len(df_A))

3585 3585


In [ ]:
df_Q = df_Q[['SENTENCE']]
df_A = df_A[['SENTENCE']]

In [ ]:
df_Q.columns = ['Q']
df_A.columns = ['A']

In [ ]:
df_Q.reset_index(inplace=True)
df_A.reset_index(inplace=True)

In [ ]:
df_Q = df_Q[['Q']]
df_A = df_A[['A']]

In [ ]:
df_Q

,Q
0,아이스아메리카노 하나요
1,네 텀블러에 넣어주세요
2,그란데 사이즈로 주세요
3,네 보고쿠폰으로 결제할게요
4,괜찮습니다
...,...
3580,영수증 하나 주실래요?
3581,텀블러 할인 있나요?
3582,반반주스 맛있어요?
3583,수제차는 직접 만드신 거예요?


In [ ]:
df_Q.reset_index(inplace=True)
df_Q

,index,Q
0,0,아이스아메리카노 하나요
1,1,네 텀블러에 넣어주세요
2,2,그란데 사이즈로 주세요
3,3,네 보고쿠폰으로 결제할게요
4,4,괜찮습니다
...,...,...
3580,3580,영수증 하나 주실래요?
3581,3581,텀블러 할인 있나요?
3582,3582,반반주스 맛있어요?
3583,3583,수제차는 직접 만드신 거예요?


In [ ]:
df_A

,A
0,테이크아웃하실 건가요?
1,텀블러 할인 300원 해드릴게요
2,적립카드 있으세요?
3,현금영수증 해드릴까요?
4,저쪽에서 잠시만 대기해주세요
...,...
3580,가지고 가시면 2000원이에요
3581,드시고 가시면 3000원이에요
3582,현금 영수증 맞으세요?
3583,"네, 여기 드릴게요"


In [ ]:
df_QA = pd.concat([df_Q,df_A], axis=1)
df_QA

,index,Q,A
0,0,아이스아메리카노 하나요,테이크아웃하실 건가요?
1,1,네 텀블러에 넣어주세요,텀블러 할인 300원 해드릴게요
2,2,그란데 사이즈로 주세요,적립카드 있으세요?
3,3,네 보고쿠폰으로 결제할게요,현금영수증 해드릴까요?
4,4,괜찮습니다,저쪽에서 잠시만 대기해주세요
...,...,...,...
3580,3580,영수증 하나 주실래요?,가지고 가시면 2000원이에요
3581,3581,텀블러 할인 있나요?,드시고 가시면 3000원이에요
3582,3582,반반주스 맛있어요?,현금 영수증 맞으세요?
3583,3583,수제차는 직접 만드신 거예요?,"네, 여기 드릴게요"


In [ ]:
train_data = df_QA

## 2.단어 집합 생성
문장 그대로 학습 모델에 넣으면 모델이 인식을 할 수 없기 때문에 단어 집합을 만들어 주어야 한다. 그리고 정수 인코딩과 패딩을 해주는 작업을 해주어야 한다.

In [ ]:
#특수기호 띄어쓰기
questions = []
for sentence in train_data['Q']:
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    questions.append(sentence)
    
    answers = []
for sentence in train_data['A']:
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    answers.append(sentence)

In [ ]:
# 서브워드텍스트인코더를 사용하여 질문, 답변 데이터로부터 단어 집합(Vocabulary) 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**13)
    
# 시작 토큰과 종료 토큰에 대한 정수 부여
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

# 시작 토큰과 종료 토큰을 고려하여 단어 집합의 크기를 + 2
VOCAB_SIZE = tokenizer.vocab_size + 2    

In [ ]:
#정수 인코딩과 패딩
# 서브워드텍스트인코더 토크나이저의 .encode()를 사용하여 텍스트 시퀀스를 정수 시퀀스로 변환.
print('임의의 질문 샘플을 정수 인코딩 : {}'.format(tokenizer.encode(questions[20])))
#출력 : 임의의 질문 샘플을 정수 인코딩 : [8656, 331]

# 최대 길이를 40으로 정의
MAX_LENGTH = 40

# 토큰화 / 정수 인코딩 / 시작 토큰과 종료 토큰 추가 / 패딩
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []

  for (sentence1, sentence2) in zip(inputs, outputs):
    # encode(토큰화 + 정수 인코딩), 시작 토큰과 종료 토큰 추가
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    tokenized_inputs.append(sentence1)
    tokenized_outputs.append(sentence2)

  # 패딩
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

  return tokenized_inputs, tokenized_outputs



임의의 질문 샘플을 정수 인코딩 : [156, 498, 23, 1]


In [ ]:
questions, answers = tokenize_and_filter(questions, answers)

샘플로 print(questions[0]), print(answers[0])을 해보면 아래와 같이 정수 인코딩과 패딩이 된 결과가 출력된다.

In [ ]:
print(questions[0]), print(answers[0])

[6410 1994  114 6411    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]
[6410 1674   64    1 6411    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]


(None, None)

마지막으로 인코더와 디코더의 입력 데이터가 되도록 배치 크기로 데이터를 묶어준다.

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1] # 디코더의 입력 / 마지막 패딩 토큰 제거
    },
    {
        'outputs': answers[:, 1:]  # 맨 처음 토큰이 제거 = 시작 토큰 제거
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)


## 3.트랜스포머 모델 만들기
위에서도 말했지만 트랜스포머를 일일이 구현하는 코드를 참고했다. https://wikidocs.net/31379 코드의 각 자세한 설명 위키독스 참조!


In [ ]:
def transformer(vocab_size, num_layers, dff,
                d_model, num_heads, dropout,
                name="transformer"):

  # 인코더의 입력
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 디코더의 입력
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  # 인코더의 패딩 마스크
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

  # 디코더의 룩어헤드 마스크(첫번째 서브층)
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask, output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

  # 디코더의 패딩 마스크(두번째 서브층)
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  # 인코더의 출력은 enc_outputs. 디코더로 전달된다.
  enc_outputs = encoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[inputs, enc_padding_mask]) # 인코더의 입력은 입력 문장과 패딩 마스크

  # 디코더의 출력은 dec_outputs. 출력층으로 전달된다.
  dec_outputs = decoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  # 다음 단어 예측을 위한 출력층
  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    # 배열의 짝수 인덱스(2i)에는 사인 함수 적용
    sines = tf.math.sin(angle_rads[:, 0::2])

    # 배열의 홀수 인덱스(2i+1)에는 코사인 함수 적용
    cosines = tf.math.cos(angle_rads[:, 1::2])

    angle_rads = np.zeros(angle_rads.shape)
    angle_rads[:, 0::2] = sines
    angle_rads[:, 1::2] = cosines
    pos_encoding = tf.constant(angle_rads)
    pos_encoding = pos_encoding[tf.newaxis, ...]

    print(pos_encoding.shape)
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [ ]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, key의 문장 길이)
  return mask[:, tf.newaxis, tf.newaxis, :]

# 디코더의 첫번째 서브층(sublayer)에서 미래 토큰을 Mask하는 함수
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x) # 패딩 마스크도 포함
  return tf.maximum(look_ahead_mask, padding_mask)

#encoder
def encoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 인코더는 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 포지셔널 인코딩 + 드롭아웃
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # 인코더를 num_layers개 쌓기
  for i in range(num_layers):
    outputs = encoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [ ]:
def encoder_layer(dff, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  # 인코더는 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 멀티-헤드 어텐션 (첫번째 서브층 / 셀프 어텐션)
  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs, 'key': inputs, 'value': inputs, # Q = K = V
          'mask': padding_mask # 패딩 마스크 사용
      })

  # 드롭아웃 + 잔차 연결과 층 정규화
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  # 포지션 와이즈 피드 포워드 신경망 (두번째 서브층)
  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 드롭아웃 + 잔차 연결과 층 정규화
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    # d_model을 num_heads로 나눈 값.
    # 논문 기준 : 64
    self.depth = d_model // self.num_heads

    # WQ, WK, WV에 해당하는 밀집층 정의
    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    # WO에 해당하는 밀집층 정의
    self.dense = tf.keras.layers.Dense(units=d_model)

  # num_heads 개수만큼 q, k, v를 split하는 함수
  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # 2. 헤드 나누기
    # q : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    # k : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
    # v : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # 3. 스케일드 닷 프로덕트 어텐션. 앞서 구현한 함수 사용.
    # (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)
    # (batch_size, query의 문장 길이, num_heads, d_model/num_heads)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 4. 헤드 연결(concatenate)하기
    # (batch_size, query의 문장 길이, d_model)
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 5. WO에 해당하는 밀집층 지나기
    # (batch_size, query의 문장 길이, d_model)
    outputs = self.dense(concat_attention)

    return outputs

In [ ]:
def scaled_dot_product_attention(query, key, value, mask):
  # query 크기 : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
  # key 크기 : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
  # value 크기 : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
  # padding_mask : (batch_size, 1, 1, key의 문장 길이)

  # Q와 K의 곱. 어텐션 스코어 행렬.
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 스케일링
  # dk의 루트값으로 나눠준다.
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # 마스킹. 어텐션 스코어 행렬의 마스킹 할 위치에 매우 작은 음수값을 넣는다.
  # 매우 작은 값이므로 소프트맥스 함수를 지나면 행렬의 해당 위치의 값은 0이 된다.
  if mask is not None:
    logits += (mask * -1e9)

  # 소프트맥스 함수는 마지막 차원인 key의 문장 길이 방향으로 수행된다.
  # attention weight : (batch_size, num_heads, query의 문장 길이, key의 문장 길이)
  attention_weights = tf.nn.softmax(logits, axis=-1)

  # output : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
  output = tf.matmul(attention_weights, value)

  return output, attention_weights

In [ ]:
def decoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')

  # 디코더는 룩어헤드 마스크(첫번째 서브층)와 패딩 마스크(두번째 서브층) 둘 다 사용.
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 포지셔널 인코딩 + 드롭아웃
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # 디코더를 num_layers개 쌓기
  for i in range(num_layers):
    outputs = decoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [ ]:
def decoder_layer(dff, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")

  # 룩어헤드 마스크(첫번째 서브층)
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")

  # 패딩 마스크(두번째 서브층)
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 멀티-헤드 어텐션 (첫번째 서브층 / 마스크드 셀프 어텐션)
  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs, 'key': inputs, 'value': inputs, # Q = K = V
          'mask': look_ahead_mask # 룩어헤드 마스크
      })

  # 잔차 연결과 층 정규화
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  # 멀티-헤드 어텐션 (두번째 서브층 / 디코더-인코더 어텐션)
  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1, 'key': enc_outputs, 'value': enc_outputs, # Q != K = V
          'mask': padding_mask # 패딩 마스크
      })

  # 드롭아웃 + 잔차 연결과 층 정규화
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  # 포지션 와이즈 피드 포워드 신경망 (세번째 서브층)
  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 드롭아웃 + 잔차 연결과 층 정규화
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [ ]:
tf.keras.backend.clear_session()

# Hyper-parameters
D_MODEL = 256
NUM_LAYERS = 2
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

(1, 6412, 256)
(1, 6412, 256)


In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)
    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

In [ ]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # 레이블의 크기는 (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

여기까지 코드 실행을 해주었다면 앞서 준비한 데이터셋으로 지정한 epochs 만큼 모델을 학습시킨다.

In [ ]:
EPOCHS = 50
model.fit(dataset, epochs=EPOCHS)

Epoch 1/50
57/57 [==============================] - 14s 64ms/step - loss: 1.2036 - accuracy: 0.0089
Epoch 2/50
57/57 [==============================] - 3s 59ms/step - loss: 1.1259 - accuracy: 0.0256
Epoch 3/50
57/57 [==============================] - 3s 54ms/step - loss: 1.0622 - accuracy: 0.0256
Epoch 4/50
57/57 [==============================] - 3s 53ms/step - loss: 0.9977 - accuracy: 0.0256
Epoch 5/50
57/57 [==============================] - 3s 54ms/step - loss: 0.9277 - accuracy: 0.0311
Epoch 6/50
57/57 [==============================] - 3s 53ms/step - loss: 0.8694 - accuracy: 0.0323
Epoch 7/50
57/57 [==============================] - 3s 54ms/step - loss: 0.8237 - accuracy: 0.0346
Epoch 8/50
57/57 [==============================] - 3s 55ms/step - loss: 0.7854 - accuracy: 0.0376
Epoch 9/50
57/57 [==============================] - 3s 53ms/step - loss: 0.7524 - accuracy: 0.0388
Epoch 10/50
57/57 [==============================] - 3s 53ms/step - loss: 0.7241 - accuracy: 0.0402
Epoch 11

위 코드를 통해 50번 학습을 시켰고, 수행 시간은 약 3분 이었다. 50번 째 학습했을 때의 loss 값은 0.0219 으로 측정되었다.

## 4.챗봇 평가하기
학습을 시켰다면 학습시킨 챗봇에 새로운 문장을 넣어서 평가를 해보고자 했다. 새로운 문장 역시 인코더 입력 형식으로 변형시켜야 하기 때문에 아래의 코드를 실행해준다.

In [ ]:
def evaluate(sentence):
  sentence = preprocess_sentence(sentence)

  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 예측 시작
  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    # 현재(마지막) 시점의 예측 단어를 받아온다.
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 마지막 시점의 예측 단어가 종료 토큰이라면 예측을 중단
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 마지막 시점의 예측 단어를 출력에 연결한다.
    # 이는 for문을 통해서 디코더의 입력으로 사용될 예정이다.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)

In [ ]:
def predict(sentence):
  prediction = evaluate(sentence)

  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('Input: {}'.format(sentence))
  print('Output: {}'.format(predicted_sentence))

  return predicted_sentence

In [ ]:
def preprocess_sentence(sentence):
  # 단어와 구두점 사이에 공백 추가.
  # ex) 12시 땡! -> 12시 땡 !
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.strip()
  return sentence

이제 predict() 함수에 문장을 입력하면 해당 문장에 대한 결과가 출력된다. 아래는 새로운 문장을 넣었을 때 출력된 결과들이다.

In [ ]:
output = predict("진동밸을 따로 주시나요??")

Input: 진동밸을 따로 주시나요??
Output: 주문번호로 불러드리겠습니다


In [ ]:
output = predict("포인트 적립해주세요")

Input: 포인트 적립해주세요
Output: 네 포인트 적립됐습니다


In [ ]:
output = predict("빵 종류도 있나요?")

Input: 빵 종류도 있나요?
Output: 네 ,  빵은 판매하지 않습니다


In [ ]:
output = predict("텀블러에 담아주세요")

Input: 텀블러에 담아주세요
Output: 텀블러 할인 300원 같이 해드릴게요


In [ ]:
output = predict("아이스 아메리카노 한 잔 주세요")

Input: 아이스 아메리카노 한 잔 주세요
Output: 사이즈 어떤 걸로 드릴까요 ?


In [ ]:
output = predict("와이파이 비밀번호는 뭔가요?")

Input: 와이파이 비밀번호는 뭔가요?
Output: 영수증 하단에 적혀있습니다


In [ ]:
output = predict("화장실은 어디에요?")

Input: 화장실은 어디에요?
Output: 화장실은 매장 밖으로 나가셔서 2층으로 가시면 됩니다


In [ ]:
output = predict("현금영수증 해주세요")

Input: 현금영수증 해주세요
Output: 네 번호 눌러주세요


In [ ]:
output = predict("마감시간은 언제인가요?")

Input: 마감시간은 언제인가요?
Output: 11시입니다


In [ ]:
output = predict("케이크 있나요?")

Input: 케이크 있나요?
Output: 리코타 멜팅 치즈 케이크 있습니다


In [ ]:
output = predict("카페모카에 휘핑크림 올라가나요?")

Input: 카페모카에 휘핑크림 올라가나요?
Output: 따뜻한 걸로 드릴까요 ?


In [ ]:
output = predict("아메리카노 아이스 주세요")

Input: 아메리카노 아이스 주세요
Output: 샌드위치는 안 필요하신가요 ?


In [ ]:
output = predict("음료 추천해주세요")

Input: 음료 추천해주세요
Output: 그럼 생과일주스가 어떠신가요


## Result

Transformer를 사용하여 카페 키오스크 챗봇을 만들어 보았다. 성능이 좋게 나온 반면에 엉뚱한 대답을 할 때도 많았다. 좀 더 많은 데이터 양을 넣어준다면 데이터를 적절히 활용하여 질문에 맞는 답변을 하는 챗봇을 만들 수 있을 것 같다. 